In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

defective_folder = 'C:\\Users\\Anthony\\Downloads\\Project_Dataset-20240405\\itmla3_project_dataset\\CONCRETE CRACKS\\Defective'
defectless_folder = 'C:\\Users\\Anthony\\Downloads\\Project_Dataset-20240405\\itmla3_project_dataset\\CONCRETE CRACKS\\Defectless'

def load_images_from_folder(folder, label):
    images =[]
    labels = []
    count = 0
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # load image in grayscale
        
        if img is not None:
            images.append(img)
            labels.append(label)
            count = count + 1
    print(count, "labeled as", label)
    return images, labels

def image_enhancement(image):
    # Apply histogram equalization
    equalized_img = cv2.equalizeHist(image)
    
    # Apply adaptive histogram equalization (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_img = clahe.apply(equalized_img)
    
    return enhanced_img

def feature_extraction(images):
    features = []
    hog = cv2.HOGDescriptor()
    
    for img in images:
        # Calculate HOG features
        hog_features = hog.compute(img)
        features.append(hog_features.flatten())
    
    return features

def feature_normalization(features):
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    return normalized_features

def train_test_split_data(features, labels, test_size=0.2, random_state=42):
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=random_state)
    return x_train, x_test, y_train, y_test

def classifier_training(x_train, y_train):
    rf_classifier = RandomForestClassifier()
    rf_classifier.fit(x_train, y_train)
    return rf_classifier

def classifier_testing(classifier, x_test, y_test):
    y_pred = classifier.predict(x_test)
    print(classification_report(y_test, y_pred))

# Load defective and defectless images and their labels
defective_images, defective_labels = load_images_from_folder(defective_folder, 1)
defectless_images, defectless_labels = load_images_from_folder(defectless_folder, 0)

# Apply image enhancement to all images
defective_images_enhanced = [image_enhancement(img) for img in defective_images]
defectless_images_enhanced = [image_enhancement(img) for img in defectless_images]

# Extract features from enhanced images
defective_features = feature_extraction(defective_images_enhanced)
defectless_features = feature_extraction(defectless_images_enhanced)

# Normalize extracted features
defective_features_normalized = feature_normalization(defective_features)
defectless_features_normalized = feature_normalization(defectless_features)

# Combine features and labels
x = defective_features_normalized + defectless_features_normalized
y = defective_labels + defectless_labels

# Split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split_data(x, y)

# Train the classifier
rf_classifier = classifier_training(x_train, y_train)

# Test the classifier
classifier_testing(rf_classifier, x_test, y_test)

plt.imshow(x[0], cmap = 'gray')
plt.show()


plt.imshow(x[19991], cmap = 'gray')
plt.show()
